# Alrighty, let's give Planet's Satellite imagery api a first test! 

### 1.0: Import the api key from a config file.

In [5]:
#Setup the API Key from the `PL_API_KEY` environment variable
from aFile import api_key
PLANET_API_KEY = api_key

### 1.1: Get our dependencies

In [20]:
import os
import json
import requests
from requests.auth import HTTPBasicAuth

### 1.2: Setup the session

In [29]:
session = requests.Session()
session.auth = (PLANET_API_KEY, '')

# 2.0: Prep the data for the map(s)

### 2.1: Load the GeoJson coordinates for the map(s) we want

In [11]:
'''
geo_json_geometry = {
  "type": "Polygon",
  "coordinates": [
    [
      [
        -122.52227783203125,
        40.660847697284815
      ],
      [
        -122.52227783203125,
        40.987154933797335
      ],
      [
        -122.01690673828124,
        40.987154933797335
      ],
      [
        -122.01690673828124,
        40.660847697284815
      ],
      [
        -122.52227783203125,
        40.660847697284815
      ]
    ]
  ]
}

In [38]:
geo_json_geometry = {
 "type": "Polygon",
 "coordinates": [
    [
      [
        -73.97801756858826,
        40.72943772441242
      ],
      [
        -73.98258805274963,
        40.73136457840986
      ],
      [
        -73.98582816123962,
        40.72696606629052
      ],
      [
        -73.98120403289795,
        40.725022823244004
      ],
      [
        -73.97801756858826,
        40.72943772441242
      ]
    ]
  ]
}

In [55]:
# filter for items the overlap with our chosen geometry
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geo_json_geometry
}

In [67]:
geometry_filter

{'type': 'GeometryFilter',
 'field_name': 'geometry',
 'config': {'type': 'Polygon',
  'coordinates': [[[-73.97801756858826, 40.72943772441242],
    [-73.98258805274963, 40.73136457840986],
    [-73.98582816123962, 40.72696606629052],
    [-73.98120403289795, 40.725022823244004],
    [-73.97801756858826, 40.72943772441242]]]}}

In [58]:
# filter images acquired in a certain date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2018-07-01T00:00:00.000Z",
    "lte": "2018-08-01T00:00:00.000Z"
  }
}

In [68]:
date_range_filter

{'type': 'DateRangeFilter',
 'field_name': 'acquired',
 'config': {'gte': '2018-07-01T00:00:00.000Z',
  'lte': '2018-08-01T00:00:00.000Z'}}

In [41]:
# filter any images which are more than 50% clouds
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.5
  }
}

In [69]:
cloud_cover_filter

{'type': 'RangeFilter', 'field_name': 'cloud_cover', 'config': {'lte': 0.5}}

In [70]:
# create a filter that combines our geo and date filters
# could also use an "OrFilter"
city1 = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}


In [71]:
city1

{'type': 'AndFilter',
 'config': [{'type': 'GeometryFilter',
   'field_name': 'geometry',
   'config': {'type': 'Polygon',
    'coordinates': [[[-73.97801756858826, 40.72943772441242],
      [-73.98258805274963, 40.73136457840986],
      [-73.98582816123962, 40.72696606629052],
      [-73.98120403289795, 40.725022823244004],
      [-73.97801756858826, 40.72943772441242]]]}},
  {'type': 'DateRangeFilter',
   'field_name': 'acquired',
   'config': {'gte': '2018-07-01T00:00:00.000Z',
    'lte': '2018-08-01T00:00:00.000Z'}},
  {'type': 'RangeFilter',
   'field_name': 'cloud_cover',
   'config': {'lte': 0.5}}]}

In [72]:
# Stats API request object
stats_endpoint_request = {
  "interval": "day",
  "item_types": ["REOrthoTile"],
  "filter": city1
}

In [73]:
stats_endpoint_request

{'interval': 'day',
 'item_types': ['REOrthoTile'],
 'filter': {'type': 'AndFilter',
  'config': [{'type': 'GeometryFilter',
    'field_name': 'geometry',
    'config': {'type': 'Polygon',
     'coordinates': [[[-73.97801756858826, 40.72943772441242],
       [-73.98258805274963, 40.73136457840986],
       [-73.98582816123962, 40.72696606629052],
       [-73.98120403289795, 40.725022823244004],
       [-73.97801756858826, 40.72943772441242]]]}},
   {'type': 'DateRangeFilter',
    'field_name': 'acquired',
    'config': {'gte': '2018-07-01T00:00:00.000Z',
     'lte': '2018-08-01T00:00:00.000Z'}},
   {'type': 'RangeFilter',
    'field_name': 'cloud_cover',
    'config': {'lte': 0.5}}]}}

In [74]:
# Search API request object
search_endpoint_request = {
  "item_types": ["REOrthoTile"],
  "filter": city1
}

In [75]:
search_endpoint_request

{'item_types': ['REOrthoTile'],
 'filter': {'type': 'AndFilter',
  'config': [{'type': 'GeometryFilter',
    'field_name': 'geometry',
    'config': {'type': 'Polygon',
     'coordinates': [[[-73.97801756858826, 40.72943772441242],
       [-73.98258805274963, 40.73136457840986],
       [-73.98582816123962, 40.72696606629052],
       [-73.98120403289795, 40.725022823244004],
       [-73.97801756858826, 40.72943772441242]]]}},
   {'type': 'DateRangeFilter',
    'field_name': 'acquired',
    'config': {'gte': '2018-07-01T00:00:00.000Z',
     'lte': '2018-08-01T00:00:00.000Z'}},
   {'type': 'RangeFilter',
    'field_name': 'cloud_cover',
    'config': {'lte': 0.5}}]}}

In [106]:
# fire off the POST request
result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=search_endpoint_request)

In [109]:
#print result.text
out_json = json.loads(result.text)
i=0
out_json

{'_links': {'_first': 'https://api.planet.com/data/v1/searches/2658e4dc40954a7cb18e186ca4ea4b12/results?_page=eyJxdWVyeV9wYXJhbXMiOiB7fSwgInNvcnRfcHJldiI6IGZhbHNlLCAicGFnZV9zaXplIjogMjUwLCAic29ydF9ieSI6ICJwdWJsaXNoZWQiLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9kZXNjIjogdHJ1ZX0%3D',
  '_next': 'https://api.planet.com/data/v1/searches/2658e4dc40954a7cb18e186ca4ea4b12/results?_page=eyJxdWVyeV9wYXJhbXMiOiB7fSwgInNvcnRfcHJldiI6IGZhbHNlLCAicGFnZV9zaXplIjogMjUwLCAic29ydF9ieSI6ICJwdWJsaXNoZWQiLCAic29ydF9zdGFydCI6ICIyMDE4LTA3LTIwVDA5OjAzOjM0LjgyMDAwMFoiLCAic29ydF9sYXN0X2lkIjogIjIwMTgwNzE5XzE1NDEwNF8xODU3ODE4X1JhcGlkRXllLTMiLCAic29ydF9kZXNjIjogdHJ1ZX0%3D',
  '_self': 'https://api.planet.com/data/v1/searches/2658e4dc40954a7cb18e186ca4ea4b12/results?_page=eyJxdWVyeV9wYXJhbXMiOiB7fSwgInNvcnRfcHJldiI6IGZhbHNlLCAicGFnZV9zaXplIjogMjUwLCAic29ydF9ieSI6ICJwdWJsaXNoZWQiLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9kZXNjIjogdHJ1ZX0%3D'},
 'features': [{'_links':

In [110]:
for feature in out_json['features']: 
	if i == 0:
		item_id = feature['id']
	i=i+1
	print(feature['id'])

20180719_154104_1857818_RapidEye-3


In [111]:
# item_id = "20160707_195147_1057916_RapidEye-1"
item_id = "20180719_154104_1857818_RapidEye-3"
item_type = "REOrthoTile"
asset_type = "visual"

In [112]:
# request an item
item = \
  session.get(
    ("https://api.planet.com/data/v1/item-types/" +
    "{}/items/{}/assets/").format(item_type, item_id))

In [113]:
print(type(item))
print(item.headers)


<class 'requests.models.Response'>
{'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Origin': '*', 'Content-Encoding': 'gzip', 'Content-Type': 'application/json', 'Date': 'Tue, 17 Sep 2019 12:48:03 GMT', 'Server': 'nginx/1.15.10', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'Vary': 'Accept-Encoding', 'X-Planet-Trace-Id': '0d38025a-d892-499e-b825-c293e483d98d', 'Via': '1.1 edge, 1.1 google', 'Alt-Svc': 'clear', 'Transfer-Encoding': 'chunked'}


In [114]:
print(item.content)

b'{"analytic": {"_links": {"_self": "https://api.planet.com/data/v1/assets/eyJpIjogIjIwMTgwNzE5XzE1NDEwNF8xODU3ODE4X1JhcGlkRXllLTMiLCAiYyI6ICJSRU9ydGhvVGlsZSIsICJ0IjogImFuYWx5dGljIiwgImN0IjogIml0ZW0tdHlwZSJ9", "activate": "https://api.planet.com/data/v1/assets/eyJpIjogIjIwMTgwNzE5XzE1NDEwNF8xODU3ODE4X1JhcGlkRXllLTMiLCAiYyI6ICJSRU9ydGhvVGlsZSIsICJ0IjogImFuYWx5dGljIiwgImN0IjogIml0ZW0tdHlwZSJ9/activate", "type": "https://api.planet.com/data/v1/asset-types/analytic"}, "_permissions": ["download"], "md5_digest": null, "status": "inactive", "type": "analytic"}, "analytic_xml": {"_links": {"_self": "https://api.planet.com/data/v1/assets/eyJpIjogIjIwMTgwNzE5XzE1NDEwNF8xODU3ODE4X1JhcGlkRXllLTMiLCAiYyI6ICJSRU9ydGhvVGlsZSIsICJ0IjogImFuYWx5dGljX3htbCIsICJjdCI6ICJpdGVtLXR5cGUifQ", "activate": "https://api.planet.com/data/v1/assets/eyJpIjogIjIwMTgwNzE5XzE1NDEwNF8xODU3ODE4X1JhcGlkRXllLTMiLCAiYyI6ICJSRU9ydGhvVGlsZSIsICJ0IjogImFuYWx5dGljX3htbCIsICJjdCI6ICJpdGVtLXR5cGUifQ/activate", "type": "https://api

In [126]:
# check if the item is active, if it is inactive, it cannot be downloaded.
# (the server has to fetch the image in the background I think.)
print(item.json()['visual']['status'])

inactive


In [127]:
# let's activate the item
# Parse out useful links
links = item.json()[u"visual"]["_links"]
self_link = links["_self"]
activation_link = links["activate"]

In [128]:
links

{'_self': 'https://api.planet.com/data/v1/assets/eyJpIjogIjIwMTgwNzE5XzE1NDEwNF8xODU3ODE4X1JhcGlkRXllLTMiLCAiYyI6ICJSRU9ydGhvVGlsZSIsICJ0IjogInZpc3VhbCIsICJjdCI6ICJpdGVtLXR5cGUifQ',
 'activate': 'https://api.planet.com/data/v1/assets/eyJpIjogIjIwMTgwNzE5XzE1NDEwNF8xODU3ODE4X1JhcGlkRXllLTMiLCAiYyI6ICJSRU9ydGhvVGlsZSIsICJ0IjogInZpc3VhbCIsICJjdCI6ICJpdGVtLXR5cGUifQ/activate',
 'type': 'https://api.planet.com/data/v1/asset-types/visual'}

In [129]:
self_link

'https://api.planet.com/data/v1/assets/eyJpIjogIjIwMTgwNzE5XzE1NDEwNF8xODU3ODE4X1JhcGlkRXllLTMiLCAiYyI6ICJSRU9ydGhvVGlsZSIsICJ0IjogInZpc3VhbCIsICJjdCI6ICJpdGVtLXR5cGUifQ'

In [130]:
activation_link

'https://api.planet.com/data/v1/assets/eyJpIjogIjIwMTgwNzE5XzE1NDEwNF8xODU3ODE4X1JhcGlkRXllLTMiLCAiYyI6ICJSRU9ydGhvVGlsZSIsICJ0IjogInZpc3VhbCIsICJjdCI6ICJpdGVtLXR5cGUifQ/activate'

In [131]:
# Request activation of the 'analytic' asset:
activate_result = \
  requests.get(
    activation_link,
    auth=HTTPBasicAuth(PLANET_API_KEY, '')
  )

In [124]:
activate_result

<Response [403]>

In [116]:
# extract the activation url from the item for the desired asset
item_activation_url = item.json()[asset_type]["location"]

## no active requests

KeyError: 'location'

In [83]:
response = session.get(item_activation_url)

In [84]:
print(response.status_code)

200


In [85]:
with open('./file03.tiff', 'wb') as f:
	for chunk in response.iter_content(1024):
		f.write(chunk)